In [1]:
%load_ext autoreload

In [ ]:
"""
Grandfather passage = ["You wish to know about my grandfather", "Well he is nearly ninety three years old", "yet he still thinks as swiftly as ever", "He dresses himself in an old black frock coat", 
               "usually several buttons missing", "A long beard clings to his chin", "giving those who observe him", "a pronounced feeling of the utmost respect",
               "When he speaks his voice is", "just a bit cracked and quivers a bit", "Twice each day he plays skillfully", "and with zest upon a small organ", "Except in the winter when",
               "the snow or ice prevents", "he slowly takes a short walk in", "the open air each day", "We have often urged him to walk", "more and smoke less",
               "but he always answers", "Banana oil Grandfather likes", "to be modern in his language"]

Each word was read character-by-character. Cue to read the characters were given 1.5 seconds apart. There are pauses between words (no pauses between characters). 
DATA is an array of (number characters, 22 - number channels, 7500 - number time steps).

Due to a glitch in the software a couple of characters are missing from some subjects. Labels are provided separately for each subject. 
"""

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from spdLearning import spdNN
from spdLearning import optimizers 
from spdLearning import trainTest
from spdLearning import spdNet

In [3]:
class BaseDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return self.data[index].astype('float32'), self.labels[index]

    def __len__(self):
        return len(self.data)

In [4]:
dev = "cpu" 
device = torch.device(dev)

In [5]:
subjectNumber = 4
subject = "Subject" + str(subjectNumber)

In [6]:
DATA = np.load("Experiment2/" + subject + "/grandfatherPassage.npy")
   
mean = np.mean(DATA, axis = -1)
std = np.std(DATA, axis = -1)
DATA = (DATA - mean[..., np.newaxis])/(std[..., np.newaxis] + 1e-5)
Labels = np.load("Experiment2/" + subject + "/grandfatherPassageLabels.npy")

In [7]:
numberChannels = 22
windowLength = 7500
numberAlphabets = 26

In [8]:
covarianceMatrices = np.zeros((len(Labels), numberChannels, numberChannels))

for j in range(len(Labels)):
    covarianceMatrices[j] = 1/windowLength * ((DATA[j] @ DATA[j].T))

In [9]:
testFeatures = covarianceMatrices
testLabels = Labels

In [10]:
testDataset = BaseDataset(testFeatures, testLabels)
testDataloader = DataLoader(testDataset, batch_size = 32, shuffle = False)

In [11]:
model = spdNet.learnSPDMatrices(numberAlphabets).to(device)
numParams = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(numParams)
lossFunction = nn.CrossEntropyLoss()
spdOptimizer = optimizers.MixOptimizer(model.parameters(), lr = 0.05)

7926


In [12]:
checkpoint = torch.load('Experiment2/' + subject + '/spdNet.pt')

In [13]:
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [14]:
def testOperationk(model, device, dataloader, Loss, k):
    with torch.no_grad():
        model.eval()
        testLoss, accuracy, correct, total = 0, 0, 0, 0
        PREDICT = []

        for data, target in dataloader:
            target = target.long()
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = Loss(output, target)
            
            testLoss += loss.data.item()
            _, topkPreds = torch.topk(output.data, k, dim = 1)
            total += target.size(0)
            correct += sum([target[i].item() in topkPreds[i].cpu().numpy() for i in range(target.size(0))])
            PREDICT.append(topkPreds.cpu().numpy())
    accuracy = 100. * correct/total
    return testLoss/total, accuracy, PREDICT

In [15]:
testLoss, testAccuracy1, prediction = testOperationk(model, device, testDataloader, lossFunction, 1)
testLoss, testAccuracy2, prediction = testOperationk(model, device, testDataloader, lossFunction, 2)
testLoss, testAccuracy3, prediction = testOperationk(model, device, testDataloader, lossFunction, 3)
testLoss, testAccuracy4, prediction = testOperationk(model, device, testDataloader, lossFunction, 4)
testLoss, testAccuracy5, prediction = testOperationk(model, device, testDataloader, lossFunction, 5)

In [16]:
print(testLoss)

0.10362897360766375


In [17]:
print("Top-1 accuracy: ", testAccuracy1)
print("Top-2 accuracy: ", testAccuracy2)
print("Top-3 accuracy: ", testAccuracy3)
print("Top-4 accuracy: ", testAccuracy4)
print("Top-5 accuracy: ", testAccuracy5)

Top-1 accuracy:  18.14814814814815
Top-2 accuracy:  29.25925925925926
Top-3 accuracy:  38.148148148148145
Top-4 accuracy:  46.111111111111114
Top-5 accuracy:  58.148148148148145


In [18]:
%autoreload